In [1]:
import os
import json
from collections import defaultdict
from tqdm import tqdm

In [2]:


def open_jsonl(path):
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            yield json.loads(line.rstrip())

folder = "/home/doyooni303/experiments/sequential_recsys/datasets/amazon"
dataset = "Home_and_Kitchen"
jsonl_path = lambda name, types: f"{name}.jsonl" if types == "data" else f"meta_{name}.jsonl"
data_path = os.path.join(folder, dataset, jsonl_path(dataset,"data"))
meta_path = os.path.join(folder, dataset, jsonl_path(dataset,"meta"))

history_dict = defaultdict(list)
itemidx2meta = dict()
item2idx, idx2item = dict(), dict()
user2idx, idx2user = dict(), dict()
uidx, iidx = 0,0
for line in open_jsonl(meta_path):
    item_id = line['parent_asin']
    if item_id not in item2idx.keys():
        item2idx.update({item_id:iidx})
        idx2item.update({iidx:item_id})
        iidx += 1

    if item_id not in itemidx2meta.keys():
        itemidx2meta.update({item2idx[item_id]:line})
for line in open_jsonl(data_path):
    user_id = line['user_id']
    iidx = item2idx[line['parent_asin']]
    review = " ".join([line['title'],line['text']]) # title과 실제 리뷰를 연결
    rating = line['rating']
    timestamp = line['timestamp']

    # histroy
    if user_id not in user2idx.keys():
        user2idx.update({user_id:uidx})
        idx2user.update({uidx:user_id})
        uidx += 1
    
    history_dict[user2idx[user_id]].append((iidx,timestamp))


filtered_history_dict = {key:value for key,value in history_dict.items() if len(value)>=3}
filtered_users_list = list(filtered_history_dict.keys())
len(filtered_history_dict)

final_history = dict()
for uidx, item_time in filtered_history_dict.items():
    item_time = sorted(item_time,key=lambda x:x[1])
    final_history.update({uidx:item_time})



100%|██████████| 67409944/67409944 [11:11<00:00, 100356.56it/s]


In [3]:
import pandas as pd
import numpy as np
num_reviews = pd.DataFrame([len(value) for value in history_dict.values()])
list_ = np.array([len(value) for value in history_dict.values()])
num_reviews.describe(),np.argmax(list_)

(                  0
 count  2.323585e+07
 mean   2.901118e+00
 std    5.243946e+00
 min    1.000000e+00
 25%    1.000000e+00
 50%    2.000000e+00
 75%    3.000000e+00
 max    2.754000e+03,
 12764)

History length가 at least >=3 이상인 user 만

7128656

In [32]:
import json
json.dump(final_history,
          open(f"./amazon/{dataset}/{dataset}_final_history.json","w"),
          indent=4,)

In [33]:
json.dump(itemidx2meta,
          open(f"./amazon/{dataset}/{dataset}_itemidx2meta.json","w"),
          indent=4,)

json.dump(user2idx,
          open(f"./amazon/{dataset}/{dataset}_user2idx.json","w"),
          indent=4,)

json.dump(item2idx,
          open(f"./amazon/{dataset}/{dataset}_item2idx.json","w"),
          indent=4,)